In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex SDK：使用预构建容器（以前称为托管运行时）训练和部署XGBoost模型

## 安装

安装最新（预览）版本的Vertex SDK。

In [ ]:
! pip3 install -U google-cloud-aiplatform --user

安装Google云存储库。

In [ ]:
! pip3 install google-cloud-storage

### 重新启动内核

一旦您安装了Vertex SDK和Google *cloud-storage*，您需要重新启动笔记本内核，以便它可以找到这些包。

In [ ]:
import os

if not os.getenv("AUTORUN"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 开始之前

### GPU 运行时

*如果您可以选择，在 GPU 运行时下运行这个笔记本。在 Colab 中，选择* **运行时 > 更改运行时类型 > GPU**

### 设置您的 GCP 项目

**无论您的笔记本环境如何，以下步骤都是必需的。**

1. [选择或创建一个 GCP 项目](https://console.cloud.google.com/cloud-resource-manager)。当您第一次创建帐户时，您会获得 300 美元的免费信用，用于支付计算/存储成本。

2. [确保为您的项目启用计费。](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [启用 Vertex API 和 Compute Engine API。](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component)

4. [Google Cloud SDK](https://cloud.google.com/sdk) 已经安装在 Google Cloud 笔记本中。

5. 在下面的单元格中输入您的项目 ID。然后运行该单元格，以确保 Cloud SDK 对这个笔记本中的所有命令使用正确的项目。

**注意**: Jupyter 以 `!` 为前缀运行行作为 shell 命令，并将以 `$` 为前缀的 Python 变量插入这些命令中。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### 地区

您还可以更改“REGION”变量，该变量在本笔记本的其余部分中使用。 以下是Vertex AI支持的地区。 我们建议在可能的情况下，选择离您最近的地区。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太地区：`asia-east1`

您不能使用多区域存储存储桶进行Vertex训练。 并非所有地区都支持所有Vertex服务。 有关每个地区的最新支持，请参阅[Vertex AI服务区域支持](https://cloud.google.com/vertex-ai/docs/general/locations)。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

如果您正在进行实时教程会话，您可能正在使用共享测试账户或项目。为了避免在创建的资源中发生名称冲突，您需要为每个实例会话创建一个时间戳，并附加到本教程中将要创建的资源的名称上。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### 认证您的GCP帐户

**如果您正在使用Google云笔记本**，则您的环境已经进行了身份验证。请跳过此步骤。

*注意：如果您正在使用Vertex笔记本并运行单元格，则单元格会自动跳过执行身份验证步骤。*

In [ ]:
import os
import sys

# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your Google Cloud account. This provides access
# to your Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# If on Vertex, then don't execute this code
if not os.path.exists("/opt/deeplearning/metadata/env_version"):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this tutorial in a notebook locally, replace the string
    # below with the path to your service account key and run this cell to
    # authenticate your Google Cloud account.
    else:
        %env GOOGLE_APPLICATION_CREDENTIALS your_path_to_credentials.json

    # Log in to your account on Google Cloud
    ! gcloud auth login

### 创建云存储桶

**无论您使用哪种笔记本环境，以下步骤都是必需的。**

本教程旨在使用公共云存储桶中的训练数据以及本地云存储桶中的批量预测数据。您也可以使用您自己在本地云存储桶中存储的训练数据。

在下面设置您的云存储桶的名称。它必须在所有云存储桶中是唯一的。

In [ ]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "aip-" + TIMESTAMP

只有在您的储存桶不存在时才运行以下单元格以创建您的云存储储存桶。

In [ ]:
! gsutil mb -l $REGION gs://$BUCKET_NAME

最后，通过检查存储桶的内容来验证对云存储桶的访问权限。

In [ ]:
! gsutil ls -al gs://$BUCKET_NAME

### 设置变量

接下来，设置一些在教程中使用的变量。
### 导入库并定义常量

#### 导入Vertex SDK

将Vertex SDK导入到我们的Python环境中。

In [ ]:
import os
import sys
import time

from google.cloud.aiplatform import gapic as aip
from google.protobuf.json_format import MessageToJson, ParseDict
from google.protobuf.struct_pb2 import Struct, Value

以下是Vertex AI 的常量设置：

- `API_ENDPOINT`：Vertex AI 用于数据集、模型、作业、流水线和端点服务的 API 服务端点。
- `PARENT`：Vertex AI 用于数据集、模型和端点资源的位置根路径。

In [ ]:
# API Endpoint
API_ENDPOINT = "{}-aiplatform.googleapis.com".format(REGION)

# Vertex AI location root path for your dataset, model and endpoint resources
PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION

## 客户端

Vertex SDK 采用客户端/服务器模型。在您的端 (Python 脚本) 中，您将创建一个客户端，用于向服务器 (Vertex) 发送请求并接收响应。

在本教程中，您将使用多个客户端，因此请提前设置它们。

- Model Service 用于管理模型。
- Endpoint Service 用于部署。
- Job Service 用于批处理作业和自定义训练。
- Prediction Service 用于提供服务。*注意*：预测具有不同的服务端点。

In [ ]:
# client options same for all services
client_options = {"api_endpoint": API_ENDPOINT}


def create_model_client():
    client = aip.ModelServiceClient(client_options=client_options)
    return client


def create_endpoint_client():
    client = aip.EndpointServiceClient(client_options=client_options)
    return client


def create_prediction_client():
    client = aip.PredictionServiceClient(client_options=client_options)
    return client


def create_job_client():
    client = aip.JobServiceClient(client_options=client_options)
    return client


clients = {}
clients["model"] = create_model_client()
clients["endpoint"] = create_endpoint_client()
clients["prediction"] = create_prediction_client()
clients["job"] = create_job_client()

for client in clients.items():
    print(client)

准备一个训练脚本。

### 包装组装

In [ ]:
# Make folder for python training script
! rm -rf custom
! mkdir custom

# Add package information
! touch custom/README.md

setup_cfg = "[egg_info]\n\
tag_build =\n\
tag_date = 0"
! echo "$setup_cfg" > custom/setup.cfg

setup_py = "import setuptools\n\
setuptools.setup(\n\
    install_requires=[\n\
    ],\n\
    packages=setuptools.find_packages())"
! echo "$setup_py" > custom/setup.py

pkg_info = "Metadata-Version: 1.0\n\
Name: Custom XGBoost Iris\n\
Version: 0.0.0\n\
Summary: Demonstration training script\n\
Home-page: www.google.com\n\
Author: Google\n\
Author-email: aferlitsch@google.com\n\
License: Public\n\
Description: Demo\n\
Platform: Vertex AI"
! echo "$pkg_info" > custom/PKG-INFO

# Make the training subfolder
! mkdir custom/trainer
! touch custom/trainer/__init__.py

### Task.py 的内容

In [ ]:
%%writefile custom/trainer/task.py
# Single Instance Training for Iris

import datetime
import os
import subprocess
import sys
import pandas as pd
import xgboost as xgb

import argparse

parser = argparse.ArgumentParser()
parser.add_argument('--model-dir', dest='model_dir',
                    default=os.getenv('AIP_MODEL_DIR'), type=str, help='Model dir.')
args = parser.parse_args()

# Download data
iris_data_filename = 'iris_data.csv'
iris_target_filename = 'iris_target.csv'
data_dir = 'gs://cloud-samples-data/ai-platform/iris'

# gsutil outputs everything to stderr so we need to divert it to stdout.
subprocess.check_call(['gsutil', 'cp', os.path.join(data_dir,
                                                    iris_data_filename),
                       iris_data_filename], stderr=sys.stdout)
subprocess.check_call(['gsutil', 'cp', os.path.join(data_dir,
                                                    iris_target_filename),
                       iris_target_filename], stderr=sys.stdout)


# Load data into pandas, then use `.values` to get NumPy arrays
iris_data = pd.read_csv(iris_data_filename).values
iris_target = pd.read_csv(iris_target_filename).values

# Convert one-column 2D array into 1D array for use with XGBoost
iris_target = iris_target.reshape((iris_target.size,))


# Load data into DMatrix object
dtrain = xgb.DMatrix(iris_data, label=iris_target)

# Train XGBoost model
bst = xgb.train({}, dtrain, 20)

# Export the classifier to a file
model_filename = 'model.bst'
bst.save_model(model_filename)


# Upload the saved model file to Cloud Storage
gcs_model_path = os.path.join(args.model_dir, model_filename)
subprocess.check_call(['gsutil', 'cp', model_filename, gcs_model_path],
    stderr=sys.stdout)


把培训脚本存储在您的云存储桶中###

In [ ]:
! rm -f custom.tar custom.tar.gz
! tar cvf custom.tar custom
! gzip custom.tar
! gsutil cp custom.tar.gz gs://$BUCKET_NAME/iris.tar.gz

训练一个模型

### [projects.locations.customJobs.create](https://cloud.google.com/vertex-ai/docs/reference/rest/v1beta1/projects.locations.trainingPipelines/create)

将上述英文文本翻译为中文：### [projects.locations.customJobs.create](https://cloud.google.com/vertex-ai/docs/reference/rest/v1beta1/projects.locations.trainingPipelines/create)

#### 请求

In [ ]:
TRAIN_IMAGE = "gcr.io/cloud-aiplatform/training/xgboost-cpu.1-1:latest"

JOB_NAME = "custom_job_XGB" + TIMESTAMP

WORKER_POOL_SPEC = [
    {
        "replica_count": 1,
        "machine_spec": {"machine_type": "n1-standard-4"},
        "python_package_spec": {
            "executor_image_uri": TRAIN_IMAGE,
            "package_uris": ["gs://" + BUCKET_NAME + "/iris.tar.gz"],
            "python_module": "trainer.task",
            "args": ["--model-dir=" + "gs://{}/{}".format(BUCKET_NAME, JOB_NAME)],
        },
    }
]

training_job = aip.CustomJob(
    display_name=JOB_NAME, job_spec={"worker_pool_specs": WORKER_POOL_SPEC}
)

print(
    MessageToJson(
        aip.CreateCustomJobRequest(parent=PARENT, custom_job=training_job).__dict__[
            "_pb"
        ]
    )
)

{
  "parent": "projects/migration-ucaip-training/locations/us-central1",
  "customJob": {
    "displayName": "custom_job_XGB20210323142337",
    "jobSpec": {
      "workerPoolSpecs": [
        {
          "machineSpec": {
            "machineType": "n1-standard-4"
          },
          "replicaCount": "1",
          "pythonPackageSpec": {
            "executorImageUri": "gcr.io/cloud-aiplatform/training/xgboost-cpu.1-1:latest",
            "packageUris": [
              "gs://migration-ucaip-trainingaip-20210323142337/iris.tar.gz"
            ],
            "pythonModule": "trainer.task",
            "args": [
              "--model-dir=gs://migration-ucaip-trainingaip-20210323142337/custom_job_XGB20210323142337"
            ]
          }
        }
      ]
    }
  }
}

#### 打电话

In [ ]:
request = clients["job"].create_custom_job(parent=PARENT, custom_job=training_job)

#### 回复

In [ ]:
print(MessageToJson(request.__dict__["_pb"]))

{
  "name": "projects/116273516712/locations/us-central1/customJobs/7371064379959148544",
  "displayName": "custom_job_XGB20210323142337",
  "jobSpec": {
    "workerPoolSpecs": [
      {
        "machineSpec": {
          "machineType": "n1-standard-4"
        },
        "replicaCount": "1",
        "diskSpec": {
          "bootDiskType": "pd-ssd",
          "bootDiskSizeGb": 100
        },
        "pythonPackageSpec": {
          "executorImageUri": "gcr.io/cloud-aiplatform/training/xgboost-cpu.1-1:latest",
          "packageUris": [
            "gs://migration-ucaip-trainingaip-20210323142337/iris.tar.gz"
          ],
          "pythonModule": "trainer.task",
          "args": [
            "--model-dir=gs://migration-ucaip-trainingaip-20210323142337/custom_job_XGB20210323142337"
          ]
        }
      }
    ]
  },
  "state": "JOB_STATE_PENDING",
  "createTime": "2021-03-23T14:23:45.067026Z",
  "updateTime": "2021-03-23T14:23:45.067026Z"
}

In [ ]:
# The full unique ID for the custom training job
custom_training_id = request.name
# The short numeric ID for the custom training job
custom_training_short_id = custom_training_id.split("/")[-1]

print(custom_training_id)

### [projects.locations.customJobs.get](https://cloud.google.com/vertex-ai/docs/reference/rest/v1beta1/projects.locations.trainingPipelines/get)

### [projects.locations.customJobs.get](https://cloud.google.com/vertex-ai/docs/reference/rest/v1beta1/projects.locations.trainingPipelines/get)

####电话

In [ ]:
request = clients["job"].get_custom_job(name=custom_training_id)

#### 回复

In [ ]:
print(MessageToJson(request.__dict__["_pb"]))

{
  "name": "projects/116273516712/locations/us-central1/customJobs/7371064379959148544",
  "displayName": "custom_job_XGB20210323142337",
  "jobSpec": {
    "workerPoolSpecs": [
      {
        "machineSpec": {
          "machineType": "n1-standard-4"
        },
        "replicaCount": "1",
        "diskSpec": {
          "bootDiskType": "pd-ssd",
          "bootDiskSizeGb": 100
        },
        "pythonPackageSpec": {
          "executorImageUri": "gcr.io/cloud-aiplatform/training/xgboost-cpu.1-1:latest",
          "packageUris": [
            "gs://migration-ucaip-trainingaip-20210323142337/iris.tar.gz"
          ],
          "pythonModule": "trainer.task",
          "args": [
            "--model-dir=gs://migration-ucaip-trainingaip-20210323142337/custom_job_XGB20210323142337"
          ]
        }
      }
    ]
  },
  "state": "JOB_STATE_PENDING",
  "createTime": "2021-03-23T14:23:45.067026Z",
  "updateTime": "2021-03-23T14:23:45.067026Z"
}

In [ ]:
while True:
    response = clients["job"].get_custom_job(name=custom_training_id)
    if response.state != aip.PipelineState.PIPELINE_STATE_SUCCEEDED:
        print("Training job has not completed:", response.state)
        if response.state == aip.PipelineState.PIPELINE_STATE_FAILED:
            break
    else:
        print("Training Time:", response.end_time - response.start_time)
        break
    time.sleep(60)

# model artifact output directory on Google Cloud Storage
model_artifact_dir = (
    response.job_spec.worker_pool_specs[0].python_package_spec.args[0].split("=")[-1]
)
print("artifact location  " + model_artifact_dir)

部署模型

[projects.locations.models.upload](https://cloud.google.com/vertex-ai/docs/reference/rest/v1beta1/projects.locations.models/upload) 的中文翻译是：[项目地点模型上传](https://cloud.google.com/vertex-ai/docs/reference/rest/v1beta1/projects.locations.models/upload)

请求

In [ ]:
DEPLOY_IMAGE = "gcr.io/cloud-aiplatform/prediction/xgboost-cpu.1-1:latest"

model = {
    "display_name": "custom_job_XGB" + TIMESTAMP,
    "artifact_uri": model_artifact_dir,
    "container_spec": {"image_uri": DEPLOY_IMAGE, "ports": [{"container_port": 8080}]},
}

print(MessageToJson(aip.UploadModelRequest(parent=PARENT, model=model).__dict__["_pb"]))

```
{
  "parent": "projects/migration-ucaip-training/locations/us-central1",
  "model": {
    "displayName": "custom_job_XGB20210323142337",
    "containerSpec": {
      "imageUri": "gcr.io/cloud-aiplatform/prediction/xgboost-cpu.1-1:latest",
      "ports": [
        {
          "containerPort": 8080
        }
      ]
    },
    "artifactUri": "gs://migration-ucaip-trainingaip-20210323142337/custom_job_XGB20210323142337"
  }
}
```

#### 呼叫

In [ ]:
request = clients["model"].upload_model(parent=PARENT, model=model)

#### 回复

In [ ]:
result = request.result()

print(MessageToJson(result.__dict__["_pb"]))

*示例输出*：
```
{
  "model": "projects/116273516712/locations/us-central1/models/2093698837704081408"
}
``` 

*示例輸出*： 
```
{
  "model": "projects/116273516712/locations/us-central1/models/2093698837704081408"
}
```

In [ ]:
# The full unique ID for the model version
model_id = result.model

## 进行批量预测

生成一批预测文件

In [ ]:
import json

import tensorflow as tf

INSTANCES = [[1.4, 1.3, 5.1, 2.8], [1.5, 1.2, 4.7, 2.4]]

gcs_input_uri = "gs://" + BUCKET_NAME + "/" + "test.jsonl"
with tf.io.gfile.GFile(gcs_input_uri, "w") as f:
    for i in INSTANCES:
        f.write(str(i) + "\n")

! gsutil cat $gcs_input_uri

示例输出:
```
[1.4, 1.3, 5.1, 2.8]
[1.5, 1.2, 4.7, 2.4]
```

### [projects.locations.batchPredictionJobs.create](https://cloud.google.com/vertex-ai/docs/reference/rest/v1beta1/projects.locations.batchPredictionJobs/create) 

### [projects.locations.batchPredictionJobs.create](https://cloud.google.com/vertex-ai/docs/reference/rest/v1beta1/projects.locations.batchPredictionJobs/create)

#### 请求

In [ ]:
model_parameters = Value(
    struct_value=Struct(
        fields={
            "confidence_threshold": Value(number_value=0.5),
            "max_predictions": Value(number_value=10000.0),
        }
    )
)

batch_prediction_job = {
    "display_name": "custom_job_XGB" + TIMESTAMP,
    "model": model_id,
    "input_config": {
        "instances_format": "jsonl",
        "gcs_source": {"uris": [gcs_input_uri]},
    },
    "model_parameters": model_parameters,
    "output_config": {
        "predictions_format": "jsonl",
        "gcs_destination": {
            "output_uri_prefix": "gs://" + f"{BUCKET_NAME}/batch_output/"
        },
    },
    "dedicated_resources": {
        "machine_spec": {"machine_type": "n1-standard-2"},
        "starting_replica_count": 1,
        "max_replica_count": 1,
    },
}

print(
    MessageToJson(
        aip.CreateBatchPredictionJobRequest(
            parent=PARENT, batch_prediction_job=batch_prediction_job
        ).__dict__["_pb"]
    )
)

{
  "parent": "projects/migration-ucaip-training/locations/us-central1",
  "batchPredictionJob": {
    "displayName": "custom_job_XGB20210323142337",
    "model": "projects/116273516712/locations/us-central1/models/2093698837704081408",
    "inputConfig": {
      "instancesFormat": "jsonl",
      "gcsSource": {
        "uris": [
          "gs://migration-ucaip-trainingaip-20210323142337/test.jsonl"
        ]
      }
    },
    "modelParameters": {
      "max_predictions": 10000.0,
      "confidence_threshold": 0.5
    },
    "outputConfig": {
      "predictionsFormat": "jsonl",
      "gcsDestination": {
        "outputUriPrefix": "gs://migration-ucaip-trainingaip-20210323142337/batch_output/"
      }
    },
    "dedicatedResources": {
      "machineSpec": {
        "machineType": "n1-standard-2"
      },
      "startingReplicaCount": 1,
      "maxReplicaCount": 1
    }
  }
}

#### 电话

In [ ]:
request = clients["job"].create_batch_prediction_job(
    parent=PARENT, batch_prediction_job=batch_prediction_job
)

回复

In [ ]:
print(MessageToJson(request.__dict__["_pb"]))

```
{
  "name": "projects/116273516712/locations/us-central1/batchPredictionJobs/1415053872761667584",
  "displayName": "custom_job_XGB20210323142337",
  "model": "projects/116273516712/locations/us-central1/models/2093698837704081408",
  "inputConfig": {
    "instancesFormat": "jsonl",
    "gcsSource": {
      "uris": [
        "gs://migration-ucaip-trainingaip-20210323142337/test.jsonl"
      ]
    }
  },
  "modelParameters": {
    "confidence_threshold": 0.5,
    "max_predictions": 10000.0
  },
  "outputConfig": {
    "predictionsFormat": "jsonl",
    "gcsDestination": {
      "outputUriPrefix": "gs://migration-ucaip-trainingaip-20210323142337/batch_output/"
    }
  },
  "dedicatedResources": {
    "machineSpec": {
      "machineType": "n1-standard-2"
    },
    "startingReplicaCount": 1,
    "maxReplicaCount": 1
  },
  "manualBatchTuningParameters": {},
  "state": "JOB_STATE_PENDING",
  "createTime": "2021-03-23T14:25:10.582704Z",
  "updateTime": "2021-03-23T14:25:10.582704Z"
}
```

In [ ]:
# The fully qualified ID for the batch job
batch_job_id = request.name
# The short numeric ID for the batch job
batch_job_short_id = batch_job_id.split("/")[-1]

print(batch_job_id)

###[projects.locations.batchPredictionJobs.get](https://cloud.google.com/vertex-ai/docs/reference/rest/v1beta1/projects.locations.batchPredictionJobs/get) 

将上述英文文本翻译成中文：[projects.locations.batchPredictionJobs.get](https://cloud.google.com/vertex-ai/docs/reference/rest/v1beta1/projects.locations.batchPredictionJobs/get)

#### 电话

In [ ]:
request = clients["job"].get_batch_prediction_job(name=batch_job_id)

####回应

In [ ]:
print(MessageToJson(request.__dict__["_pb"]))

*示例输出*：
```
{
  "name": "projects/116273516712/locations/us-central1/batchPredictionJobs/1415053872761667584",
  "displayName": "custom_job_XGB20210323142337",
  "model": "projects/116273516712/locations/us-central1/models/2093698837704081408",
  "inputConfig": {
    "instancesFormat": "jsonl",
    "gcsSource": {
      "uris": [
        "gs://migration-ucaip-trainingaip-20210323142337/test.jsonl"
      ]
    }
  },
  "modelParameters": {
    "max_predictions": 10000.0,
    "confidence_threshold": 0.5
  },
  "outputConfig": {
    "predictionsFormat": "jsonl",
    "gcsDestination": {
      "outputUriPrefix": "gs://migration-ucaip-trainingaip-20210323142337/batch_output/"
    }
  },
  "dedicatedResources": {
    "machineSpec": {
      "machineType": "n1-standard-2"
    },
    "startingReplicaCount": 1,
    "maxReplicaCount": 1
  },
  "manualBatchTuningParameters": {},
  "state": "JOB_STATE_PENDING",
  "createTime": "2021-03-23T14:25:10.582704Z",
  "updateTime": "2021-03-23T14:25:10.582704Z"
}
```

In [ ]:
def get_latest_predictions(gcs_out_dir):
    """ Get the latest prediction subfolder using the timestamp in the subfolder name"""
    folders = !gsutil ls $gcs_out_dir
    latest = ""
    for folder in folders:
        subfolder = folder.split("/")[-2]
        if subfolder.startswith("prediction-"):
            if subfolder > latest:
                latest = folder[:-1]
    return latest


while True:
    response = clients["job"].get_batch_prediction_job(name=batch_job_id)
    if response.state != aip.JobState.JOB_STATE_SUCCEEDED:
        print("The job has not completed:", response.state)
        if response.state == aip.JobState.JOB_STATE_FAILED:
            break
    else:
        folder = get_latest_predictions(
            response.output_config.gcs_destination.output_uri_prefix
        )
        ! gsutil ls $folder/prediction*

        ! gsutil cat -h $folder/prediction*
        break
    time.sleep(60)

*示例输出*：
```
==> gs://migration-ucaip-trainingaip-20210323142337/batch_output/prediction-custom_job_XGB20210323142337-2021_03_23T07_25_10_544Z/prediction.errors_stats-00000-of-00001 <==

==> gs://migration-ucaip-trainingaip-20210323142337/batch_output/prediction-custom_job_XGB20210323142337-2021_03_23T07_25_10_544Z/prediction.results-00000-of-00001 <==
{"instance": [1.4, 1.3, 5.1, 2.8], "prediction": 2.0451931953430176}
{"instance": [1.5, 1.2, 4.7, 2.4], "prediction": 1.9618644714355469}
```

做在线预测

### [projects.locations.endpoints.create](https://cloud.google.com/vertex-ai/docs/reference/rest/v1beta1/projects.locations.endpoints/create)

### [projects.locations.endpoints.create](https://cloud.google.com/vertex-ai/docs/reference/rest/v1beta1/projects.locations.endpoints/create)

请求

In [ ]:
endpoint = {"display_name": "custom_job_XGB" + TIMESTAMP}

print(
    MessageToJson(
        aip.CreateEndpointRequest(parent=PARENT, endpoint=endpoint).__dict__["_pb"]
    )
)

{
  "parent": "projects/migration-ucaip-training/locations/us-central1",
  "endpoint": {
    "displayName": "custom_job_XGB20210323142337"
  }
}
```的示例输出：

#### 电话

In [ ]:
request = clients["endpoint"].create_endpoint(parent=PARENT, endpoint=endpoint)

反应

In [ ]:
result = request.result()

print(MessageToJson(result.__dict__["_pb"]))

{
  "name": "projects/116273516712/locations/us-central1/endpoints/1733903448723685376"
}
```

中文翻译：
```
{
  "name": "projects/116273516712/locations/us-central1/endpoints/1733903448723685376"
}
```

In [ ]:
# The full unique ID for the endpoint
endpoint_id = result.name
# The short numeric ID for the endpoint
endpoint_short_id = endpoint_id.split("/")[-1]

print(endpoint_id)

### [projects.locations.endpoints.deployModel](https://cloud.google.com/vertex-ai/docs/reference/rest/v1beta1/projects.locations.endpoints/deployModel)

### [projects.locations.endpoints.deployModel](https://cloud.google.com/vertex-ai/docs/reference/rest/v1beta1/projects.locations.endpoints/deployModel)

请求

In [ ]:
deployed_model = {
    "model": model_id,
    "display_name": "custom_job_XGB" + TIMESTAMP,
    "dedicated_resources": {
        "min_replica_count": 1,
        "max_replica_count": 1,
        "machine_spec": {"machine_type": "n1-standard-4", "accelerator_count": 0},
    },
}

print(
    MessageToJson(
        aip.DeployModelRequest(
            endpoint=endpoint_id,
            deployed_model=deployed_model,
            traffic_split={"0": 100},
        ).__dict__["_pb"]
    )
)

{
  "endpoint": "projects/116273516712/locations/us-central1/endpoints/1733903448723685376",
  "deployedModel": {
    "model": "projects/116273516712/locations/us-central1/models/2093698837704081408",
    "displayName": "custom_job_XGB20210323142337",
    "dedicatedResources": {
      "machineSpec": {
        "machineType": "n1-standard-4"
      },
      "minReplicaCount": 1,
      "maxReplicaCount": 1
    }
  },
  "trafficSplit": {
    "0": 100
  }
}

呼叫

In [ ]:
request = clients["endpoint"].deploy_model(
    endpoint=endpoint_id, deployed_model=deployed_model, traffic_split={"0": 100}
)

### 回复

In [ ]:
result = request.result()

print(MessageToJson(result.__dict__["_pb"]))

*示例输出*：
```
{
  "deployedModel": {
    "id": "7407594554280378368"
  }
}
```
Translated to Chinese:
```
{
  "deployedModel": {
    "id": "7407594554280378368"
  }
}
```

In [ ]:
# The unique ID for the deployed model
deployed_model_id = result.deployed_model.id

print(deployed_model_id)

### [projects.locations.endpoints.predict](https://cloud.google.com/vertex-ai/docs/reference/rest/v1beta1/projects.locations.endpoints/predict)
### [项目.地点.端点.预测](https://cloud.google.com/vertex-ai/docs/reference/rest/v1beta1/projects.locations.endpoints/predict)

准备文件用于在线预测

In [ ]:
INSTANCES = [[1.4, 1.3, 5.1, 2.8], [1.5, 1.2, 4.7, 2.4]]

#### 请求

In [ ]:
prediction_request = {"endpoint": endpoint_id, "instances": INSTANCES}

print(json.dumps(prediction_request, indent=2))

*示例输出*：
```
{
  "端点": "projects/116273516712/locations/us-central1/endpoints/1733903448723685376",
  "实例": [
    [
      1.4,
      1.3,
      5.1,
      2.8
    ],
    [
      1.5,
      1.2,
      4.7,
      2.4
    ]
  ]
}
```

电话

In [ ]:
request = clients["prediction"].predict(endpoint=endpoint_id, instances=INSTANCES)

#### 响应

In [ ]:
print(MessageToJson(request.__dict__["_pb"]))

```
{
  "预测结果": [
    2.045193195343018,
    1.961864471435547
  ],
  "部署的模型ID": "7407594554280378368"
}
```

### [projects.locations.endpoints.undeployModel](https://cloud.google.com/vertex-ai/docs/reference/rest/v1beta1/projects.locations.endpoints/undeployModel)

### [projects.locations.endpoints.undeployModel](https://cloud.google.com/vertex-ai/docs/reference/rest/v1beta1/projects.locations.endpoints/undeployModel)

召唤

In [ ]:
request = clients["endpoint"].undeploy_model(
    endpoint=endpoint_id, deployed_model_id=deployed_model_id, traffic_split={}
)

####回复

In [ ]:
result = request.result()

print(MessageToJson(result.__dict__["_pb"]))

*示例输出*：
```
{}
```

清理

要清理本项目中使用的所有GCP资源，您可以删除用于本教程的[GCP项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的各个资源。

In [ ]:
delete_model = True
delete_endpoint = True
delete_pipeline = True
delete_batchjob = True
delete_bucket = True

# Delete the model using the Vertex AI fully qualified identifier for the model
try:
    if delete_model:
        clients["model"].delete_model(name=model_id)
except Exception as e:
    print(e)

# Delete the endpoint using the Vertex AI fully qualified identifier for the endpoint
try:
    if delete_endpoint:
        clients["endpoint"].delete_endpoint(name=endpoint_id)
except Exception as e:
    print(e)

# Delete the custom training using the Vertex AI fully qualified identifier for the custome training
try:
    if custom_training_id:
        clients["job"].delete_custom_job(name=custom_training_id)
except Exception as e:
    print(e)

# Delete the batch job using the Vertex AI fully qualified identifier for the batch job
try:
    if delete_batchjob:
        clients["job"].delete_batch_prediction_job(name=batch_job_id)
except Exception as e:
    print(e)

if delete_bucket and "BUCKET_NAME" in globals():
    ! gsutil rm -r gs://$BUCKET_NAME